In [ ]:
import torch
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts import PromptTemplate

selected_model = "mistralai/Mixtral-8x7B-Instruct-v0.1"

SYSTEM_PROMPT = """You are an AI assistant that answers questions in a friendly manner, based on the given source documents. Here are some rules you always follow:
- Generate human readable output, avoid creating output with gibberish text.
- Generate only the requested output, don't include any other language before or after the requested output.
- Never say thank you, that you are happy to help, that you are an AI agent, etc. Just answer directly.
- Generate professional language typically used in business documents in North America.
- Never generate offensive or foul language.
"""

query_wrapper_prompt = PromptTemplate(
    "Context information is below.\\n"
    "---------------------\\n"
    "{context_str}\\n"
    "---------------------\\n"
    "Given the context information and not prior knowledge, "
    "answer the query. Please be brief, concise, and complete.\\n"
    "If the context information does not contain an answer to the query, "
    "respond with \"No information\"."
    "Query: {query_str}\\n"
    "Answer: "
)

llm = HuggingFaceLLM(
    context_window=30000,
    max_new_tokens=2048,
    query_wrapper_prompt=query_wrapper_prompt,
    model_name=selected_model,
    tokenizer_name=selected_model,
    device_map="auto",
    # change these settings below depending on your GPU
    model_kwargs={"torch_dtype": torch.float16, "load_in_8bit": False, "trust_remote_code":True},
)

In [ ]:
from huggingface_hub import login
login("*****")

In [ ]:
from llama_index.embeddings import HuggingFaceEmbedding
# Replace 'your-huggingface-model' with the actual model name from Hugging Face
embed_model = HuggingFaceEmbedding(model_name="jinaai/jina-embeddings-v2-base-en")

In [ ]:
import time 
import pypdf 
import pandas as pd 
from llama_index.evaluation import ( 
    RelevancyEvaluator, 
    FaithfulnessEvaluator, 
) 

from llama_index import ( 
    SimpleDirectoryReader, 
    VectorStoreIndex, 
    ServiceContext,
    set_global_service_context
)

documents = SimpleDirectoryReader("../data/").load_data()
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:jina-embeddings-v2-base-en"
)
set_global_service_context(service_context)
index = VectorStoreIndex.from_documents(  
    documents=documents
)

In [ ]:
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index import get_response_synthesizer

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=2,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    service_context=service_context,
    text_qa_template=query_wrapper_prompt,
    response_mode="compact",
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

In [ ]:
result = query_engine.query("What is Multimodal Agents?")
print(result.response)

In [ ]:
import time

response = query_engine.query("What is Multimodal Agents?")

start_time = time.time()

token_count = 0
for token in response.response_gen:
    print(token, end="")
    token_count += 1

time_elapsed = time.time() - start_time
tokens_per_second = token_count / time_elapsed

print(f"\n\nStreamed output at {tokens_per_second} tokens/s")